# Interactive structures with nglview
The [pdb_prot_align.nglview_struct](https://jbloomlab.github.io/pdb_prot_align/pdb_prot_align.nglview_struct.html) module contains functions that make it easy to manipulate structures using in [nglview](http://nglviewer.org/nglview/latest/index.html).
Note that [nglview](http://nglviewer.org/nglview/latest/index.html) is designed for interactive use in a Jupyter notebook, so this example only works if run interactively.

This example shows how to color the structure of the CoV Spike, using a measure of alignment variability created by running `pdb_prot_align` in one of the earlier examples.
So we first specify the paths of the PDB file and the `*_sites.csv` file created by `pdb_prot_align`:

In [1]:
pdbfile = 'input_files/6crv.pdb'
sites_csv = 'output_files/beta_coronavirus_Spike_sites.csv'

First we import the necessary Python modules, which include [nglview](http://nglviewer.org/nglview/latest/index.html), which you must ensure is [installed](http://nglviewer.org/nglview/latest/index.html#installation).
We also import [pdb_prot_align.colorschemes](https://jbloomlab.github.io/pdb_prot_align/pdb_prot_align.colorschemes) and [pdb_prot_align.nglview_struct](https://jbloomlab.github.io/pdb_prot_align/pdb_prot_align.nglview_struct.html) modules:

In [2]:
import pandas as pd

import nglview

import pdb_prot_align

import pdb_prot_align.colorschemes
import pdb_prot_align.nglview_struct

_ColormakerRegistry()

We are going to color the structure according to the number of effective amino acids, so we get the maximum number at any site to set the color scale limits (note that it would also be reasonable to color by entropy):

In [ ]:
max_neff = pd.read_csv(sites_csv)['n_effective'].max()
print(f"Max number of effective amino acids is {max_neff:.2f}")

Create a [ValueToColorMap](https://jbloomlab.github.io/pdb_prot_align/pdb_prot_align.colorschemes.html#pdb_prot_align.colorschemes.ValueToColorMap) to map the numerical values to colors:

In [ ]:
color_map = pdb_prot_align.colorschemes.ValueToColorMap(minvalue=1,
                                                        maxvalue=max_neff,
                                                        cmap='viridis')

_ = color_map.scale_bar(label='effective amino acids', orientation='horizontal')

In [3]:
view = nglview.show_structure_file(pdbfile,
                                   default_representation=True)

In [6]:
view
view.clear()
view.add_cartoon()
view

NGLWidget()

In [5]:
nglview.__version__

'2.7.1'